# <u>Manejo de datos con Pandas</u>

## Datos ordenados

In [ ]:
import pandas as pd
import numpy as np

### 1. Valores como cabeceras de las columnas en lugar de nombres de variables  

In [ ]:
incomes_per_religion = pd.read_csv("data/religion_income.csv")
incomes_per_religion.head()

Para pasar de una tabla dinámica a una tabla ordenada (tidy), podemos utilizar **melt**:

In [ ]:
incomes_per_religion_tidy = pd.melt(incomes_per_religion, 
                                    id_vars=['religion'], 
                                    var_name='income', 
                                    value_name='frequency')
incomes_per_religion_tidy.set_index('religion',inplace=True)
incomes_per_religion_tidy.sort_index().head(10)

In [ ]:
incomes_per_religion_tidy.sort_index().head(20)

### 2. Varias variables se almacenan en un sola columna

In [ ]:
tb = pd.read_csv("data/tb.csv")
tb.shape

In [ ]:
tb.head()

In [ ]:
tb_melt = pd.melt(tb, id_vars=['iso2', 'year'])
tb_melt.set_index('iso2',inplace=True)
tb_melt.sort_index()
tb_melt.head()

In [ ]:
tb_melt['gender'] = tb_melt.variable.str[0]
tb_melt['age_group'] = tb_melt.variable.str[1:]
tb_melt.head()

In [ ]:
tb_melt.drop(columns='variable',inplace=True)
tb_melt.head()

In [ ]:
tb_melt = tb_melt.dropna(how='any')
tb_melt.head(20)

### 3. Variables tanto en filas como en columnas

In [ ]:
weather = pd.read_csv("data/weather.csv")
weather.head()

In [ ]:
weather_molten = pd.melt(weather,id_vars=['id','year','month','element'], var_name='day', value_name='value')
weather_molten.head()

In [ ]:
weather_molten.day= (weather_molten.day.str.replace("d","")).astype(np.int64)
weather_molten['date'] = pd.to_datetime(weather_molten[['year','month','day']],errors ='coerce')
weather_molten[weather_molten.date == 'NaT'].dropna() # Eliminando los NaT
weather_molten.sort_values(by='date',inplace=True)
weather_molten.head()

In [ ]:
weather_molten.head(120)

In [ ]:
weather_molten_pivot = weather_molten.pivot_table(index=["id","date"], columns="element", values="value")
weather_molten_pivot.sort_values(by='date',inplace=True)
weather_molten_pivot.head(20)

### 4. Resultados de varios tipos de experimentos en una misma tabla

In [ ]:
billboard = pd.read_excel("data/billboard.xlsx")
billboard.head()

In [ ]:
songs_cols = ["year", "artist.inverted", "track", "time", "genre"]
songs = billboard[songs_cols].drop_duplicates() # Eliminamos posibles duplicados
songs = songs.reset_index(drop=True) # Se reinicia el índice eliminando el antiguo
songs["song_id"] = songs.index
songs.head()

In [ ]:
songs.shape

In [ ]:
songs.tail(10)

In [ ]:
# Usando melt para obtener el ranking cada semana
id_vars = ["year","artist.inverted","track","time","genre","date.entered","date.peaked"]

df = pd.melt(frame=billboard,id_vars=id_vars, var_name="week", value_name="rank")
df

In [ ]:
# Dando formato a la columna semana
df["week"] = df['week'].str.extract('(\d+)', expand=False).astype(int)
df.head()

In [ ]:
# Eliminando las filas con valore nulos
df = df.dropna(how='any')
df

In [ ]:
df.tail()

In [ ]:
# Creando un columna "date" para la fecha de cada semana en el ranking
df['date'] = pd.to_datetime(df['date.entered']) + pd.to_timedelta(df['week'], unit='w') - pd.DateOffset(weeks=1)
df.head()

In [ ]:
df

In [ ]:
df[df.track == 'Maria, Maria']

In [ ]:
df[df.track == "Toca's Miracle"]

In [ ]:
# Seleccionando solo las columnas que nos interesan
billboard2= df[["year","artist.inverted","track","time","genre","week","rank","date"]]
billboard2 = billboard2.sort_values(ascending=True, by=["year","artist.inverted","track","week","rank"])
billboard2.head()

In [ ]:
# Tabla final con el ranking de cada canción por semana
ranks = pd.merge(billboard2, songs, on=["year","artist.inverted", "track", "time", "genre"])
ranks = ranks[["song_id", "date","rank"]]
ranks.head(20)

## Data Cleaning

### 1. Valores perdidos

In [ ]:
import pandas as pd

d_students = pd.read_csv('data/students.csv')

In [ ]:
d_students.info()

In [ ]:
# Identificar a los valores perdidos.
d_students['REGION'].isnull()

In [ ]:
# Identificar a los valores no perdidos.
d_students['PCT OVERWEIGHT'].notnull()

In [ ]:
# Cuantificar a los valores perdidos.
d_students['PCT OVERWEIGHT'].isnull().value_counts()

In [ ]:
# Cuantificar a los valores no perdidos.
d_students['PCT OVERWEIGHT'].notnull().value_counts()

In [ ]:
d_students['PCT OVERWEIGHT'].head(20)

In [ ]:
# Eliminar a los valores perdidos (NaN).
d_students['PCT OVERWEIGHT'].dropna().head(20)

In [ ]:
# Método alternativo para eliminar nulos.
d_students['PCT OVERWEIGHT'].replace(np.nan,'0%').head(20)

In [ ]:
d_students.head(20)

In [ ]:
# Eliminar a cualquier registro que tenga por lo menos un campo con valor perdido: how='any'
# Para eliminar filas completas con valores todos los valores vacíos: how='all'
d = d_students.dropna(how='any')

In [ ]:
d.head(15)

In [ ]:
d.reset_index(inplace=True)

In [ ]:
d.head(15)

In [ ]:
d = d.drop(columns = 'index')

In [ ]:
d.head(15)

In [ ]:
# Cuantificar a los valores perdidos.
d['PCT OVERWEIGHT'].isnull().value_counts()

In [ ]:
# detectar preliminarmente nulos en mi base de datos
d.info()

### 2. Imputar Valores Perdidos

In [ ]:
import numpy as np

# Crear un data frame en base a números aleatorios.
df = pd.DataFrame(np.random.randn(5, 3), index=['a0', 'a10','a20', 'a30', 'a40'],columns=['X', 'Y', 'Z'])
df

In [ ]:
# Crear índices adicionales al data frame.
df2 = df.reindex(['a0', 'a1', 'a10', 'a11', 'a20', 'a21','a30', 'a31', 'a40', 'a41'])
df2

In [ ]:
# Completar los valores perdidos con ceros.
df3 = df2.fillna(0)
df3

In [ ]:
# Completar los valores con valores diferentes por variable mediante un diccionario
values = {'X': 10, 'Y': 20, 'Z': 30}
df4 = df2.fillna(values)
df4

In [ ]:
# Completar los valores perdidos con el método “forward propagation”. Se va completar con el valor previo al nulo.
df5 = df2.fillna(method='pad')
df5

In [ ]:
# Completar los valores perdidos con el promedio de la variable.
df6 = df2.fillna(df2.mean())
df6

In [ ]:
# Completar los valores perdidos con la mediana de determinadas variables.
df7 = df2[['X','Y']].fillna(df2[['X','Y']].median())
df7

In [ ]:
# Completar los valores perdidos con la mediana de determinadas variables.
df8 = df2.fillna(df2[['X','Y']].median())
df8